In [132]:
def to_bytes(msg, byte_order='little', length=1):
    if isinstance(msg, int) or isinstance(msg, bool):
        return msg.to_bytes(length, byte_order)
    elif isinstance(msg, str):
        return msg.encode(encoding='UTF-8', errors='strict')
    elif isinstance(msg, bytes):
        return msg
    else:
        return print("message of type %s not supported by to_bytes()" % type(msg))

In [139]:
def pad_msg_end(msg, length):
    len_diff = length - len(msg)
    msg += b'\x00' * len_diff
    return msg

In [146]:
# test string
b_str = to_bytes('version')
print(b_str)
pad_msg_end(b_str, 12)

b'version'


b'version\x00\x00\x00\x00\x00'

In [147]:
# test int
b_int = to_bytes(4, length=2, byte_order='little')
print(b_int)
pad_msg_end(b_int, 10)

b'\x04\x00'


b'\x04\x00\x00\x00\x00\x00\x00\x00\x00\x00'

In [148]:
# test bool
b_bool = to_bytes(bool(1))
print(b_bool)

b'\x01'


In [149]:
# test bytes
b_bytes = to_bytes(b'\x0c\x00\x00\x00')
print(b_bytes)

b'\x0c\x00\x00\x00'


In [150]:
# test list
to_bytes([1, 2, 3])

message of type <class 'list'> not supported by to_bytes()


In [153]:
test_message = b"".join([b_str, b_int, b_bool, b_bytes])
print(test_message)

b'version\x04\x00\x01\x0c\x00\x00\x00'


In [159]:
import hashlib

th = hashlib.sha256(test_message).digest()
print(th)
h = hashlib.sha256(th).digest()
print(h)
print(h[:4])
test_message_plus_checksum = b"".join([test_message, h[:4]])
print(test_message_plus_checksum)

b'g\x8a\xd8d\xdf\x8adWP\xd4/\xa3\x87\xac\xc8q\xee\xef}q\xb8\xc0\x1d\x89\xf33\xe6\x13\xbdc7\xb3'
b'\x9f\xfd\x9fB\x85\xd2\xffgf\x99w|\xcdg\xd7>\xc8\xe5d\xd7L\xa6\x0c|N\xdf\x8cFR]i\xe7'
b'\x9f\xfd\x9fB'
b'version\x04\x00\x01\x0c\x00\x00\x00\x9f\xfd\x9fB'


In [190]:
import chainparams

def generate_header(
        StartString=to_bytes(chainparams.mainParams.StartString, length=4),
        CommandName= "",
        payload_size=4,
                    ):
    
    cmd = pad_msg_end(to_bytes(CommandName), 12)
    payload = pad_msg_end(to_bytes(payload_size), 4)

    body = b''.join([StartString, cmd, payload])

    checksum_hash1 = hashlib.sha256(body).digest()
    checksum_hash2 = hashlib.sha256(checksum_hash1).digest()

    header = b''.join([body, checksum_hash2])

    return header

In [192]:
generate_header(CommandName='version', payload_size=128)

b'\xd9\xb4\xbe\xf9version\x00\x00\x00\x00\x00\x80\x00\x00\x00\xa8\n\xc9\x7f\xdcW\xf2\xf3h\xc3\xf333C\xcejZ\x10\xfav\x07\xd1\xa2`\x08)\xb5>n\x93L\x0b'